In [ ]:
import tensorflow as tf
import numpy as np
import random
import os
import pickle
import gc

# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Optional: disable eager execution (if using graph mode code)
#tf.compat.v1.disable_eager_execution()

# Session config
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
sess = tf.compat.v1.Session(config=session_conf)

# Set this session as default for everything that follows
# No set_session needed — use a context manager instead
with sess.as_default():
    # your model/code here
    pass

tf.__version__

In [ ]:
os.getcwd()

In [ ]:
os.chdir('/mnt/d/GitHub/WQU-Capstone/notebooks/classes')

os.getcwd()

### Import Classes

In [ ]:
%load_ext autoreload
%aimport class_ForecastingTrader 
%autoreload 1

In [ ]:
# series_analyser = class_SeriesAnalyser.SeriesAnalyser()
# trader = class_Trader.Trader()
# data_processor = class_DataProcessor.DataProcessor()
forecasting_trader = class_ForecastingTrader.ForecastingTrader()

In [ ]:
current_path='/mnt/d/GitHub/WQU-Capstone/notebooks/CAE_75etf_17-5-25'
with open(f'{current_path}/cae_10_clusters_pairs_category.pickle', 'rb') as handle: 
    pairs = pickle.load(handle)
len(pairs)

## model train

In [ ]:
n_in_set = [24, 12, 6]
hidden_nodes_set = [[50], [30], [20], [10]]

hidden_nodes_names = [str(nodes[0])+'*2' if len(nodes) > 1 else str(nodes[0]) for nodes in hidden_nodes_set]

In [ ]:
for input_dim in n_in_set:
    for i, hidden_nodes in enumerate(hidden_nodes_set):
        print(f'{i}, input_dim: {input_dim}, hidden_nodes: {hidden_nodes}')
        model_config = {"n_in": input_dim,
                        "n_out": 1,
                        "epochs": 500,
                        "hidden_nodes": hidden_nodes,
                        "loss_fct": "mse",
                        "optimizer": "rmsprop",
                        "batch_size": 64,#256,
                        "train_val_split": '2023-01-01',
                        "test_init": '2024-01-01',}
        models = forecasting_trader.train_models(pairs, model_config, model_type='rnn')
        # save models for this configuration
        with open(f'{current_path}/models/lstm/models_n_in-'+str(input_dim)+'_hidden_nodes-'+hidden_nodes_names[i]+'.pkl', 'wb') as f:
            pickle.dump(models, f)

gc.collect()